Set your drive path here

In [95]:
drive_path = '/media/stephen/Brain20191/allen-brain-observatory/visual-coding-2p/'

In [96]:
import os
import pandas as pd
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import allensdk.brain_observatory.stimulus_info as stim_info
manifest_file = os.path.join(drive_path,'manifest.json')
boc = BrainObservatoryCache(manifest_file=manifest_file)


Print the various values for brain area, layer, celltype

In [97]:
brain_areas = boc.get_all_targeted_structures()
brain_areas

['VISal', 'VISam', 'VISl', 'VISp', 'VISpm', 'VISrl']

In [98]:
depths = boc.get_all_imaging_depths()
depths

[175,
 185,
 195,
 200,
 205,
 225,
 250,
 265,
 275,
 276,
 285,
 300,
 320,
 325,
 335,
 350,
 365,
 375,
 390,
 400,
 550,
 570,
 625]

In [99]:
cell_types = boc.get_all_reporter_lines()
cell_types

['Ai148(TIT2L-GC6f-ICL-tTA2)',
 'Ai162(TIT2L-GC6s-ICL-tTA2)',
 'Ai93(TITL-GCaMP6f)',
 'Ai93(TITL-GCaMP6f)-hyg',
 'Ai94(TITL-GCaMP6s)']

Experiment counts by brain area, layer, reporter line

In [100]:
exps = boc.get_ophys_experiments(stimuli=[stim_info.DRIFTING_GRATINGS])
exps_df = pd.DataFrame.from_dict(exps)
exps_df.groupby(['targeted_structure', 'imaging_depth', 'reporter_line'])['id'].count()

targeted_structure  imaging_depth  reporter_line             
VISal               175            Ai93(TITL-GCaMP6f)             9
                    265            Ai93(TITL-GCaMP6f)             1
                    275            Ai148(TIT2L-GC6f-ICL-tTA2)     1
                                   Ai93(TITL-GCaMP6f)            14
                    276            Ai93(TITL-GCaMP6f)             1
                    300            Ai93(TITL-GCaMP6f)             1
                    350            Ai93(TITL-GCaMP6f)             5
                    365            Ai93(TITL-GCaMP6f)             1
                    375            Ai93(TITL-GCaMP6f)             8
VISam               175            Ai93(TITL-GCaMP6f)             8
                    275            Ai93(TITL-GCaMP6f)            15
                    320            Ai93(TITL-GCaMP6f)             1
                    325            Ai93(TITL-GCaMP6f)             1
                    350            Ai93(TITL-GCaMP6f) 

Get cells given brain area, depth, cell type

In [101]:
cells = boc.get_cell_specimens()
cells = pd.DataFrame.from_records(cells)

def get_cells(brain_area, depth, cell_type):
    exps = boc.get_ophys_experiments(stimuli=[stim_info.DRIFTING_GRATINGS],
        targeted_structures = [brain_area],
        imaging_depths = [depth],
        reporter_lines = [cell_type])
    # Can't find [targeted_structure, reporter_line] in cell table, so have to combine w/ exps.
    exps_df = pd.DataFrame.from_dict(exps)
    if len(exps_df)== 0:
        return []
    is_in_relevant_container_mask = False * len(cells)
    for relevant_container_id in (exps_df.experiment_container_id.values):
        is_in_relevant_container_mask |= (cells.experiment_container_id == relevant_container_id)
    return cells[is_in_relevant_container_mask]
#cells_df = get_cells('VISl', 175, 'Ai148(TIT2L-GC6f-ICL-tTA2)')
#cells_df


Get cell counts per brain area, depth, cell type

In [102]:
exps = boc.get_ophys_experiments(stimuli=[stim_info.DRIFTING_GRATINGS])
exps_df = pd.DataFrame.from_dict(exps)
for brain_area in brain_areas:
    for depth in depths:
        for cell_type in boc.get_all_reporter_lines():
            if len(exps_df[
                (exps_df['targeted_structure'] == brain_area) &
                (exps_df['imaging_depth'] == depth) &
                (exps_df['reporter_line'] == cell_type)
                ]) == 0:
                continue
            cell_count = len(get_cells(brain_area, depth, cell_type))
            if cell_count == 0:
                continue
            print (brain_area, depth, cell_type, cell_count)

VISal 175 Ai93(TITL-GCaMP6f) 1802
VISal 265 Ai93(TITL-GCaMP6f) 346
VISal 275 Ai148(TIT2L-GC6f-ICL-tTA2) 24
VISal 275 Ai93(TITL-GCaMP6f) 3709
VISal 276 Ai93(TITL-GCaMP6f) 212
VISal 300 Ai93(TITL-GCaMP6f) 78
VISal 350 Ai93(TITL-GCaMP6f) 142
VISal 365 Ai93(TITL-GCaMP6f) 162
VISal 375 Ai93(TITL-GCaMP6f) 716
VISam 175 Ai93(TITL-GCaMP6f) 1021
VISam 275 Ai93(TITL-GCaMP6f) 1927
VISam 320 Ai93(TITL-GCaMP6f) 5
VISam 325 Ai93(TITL-GCaMP6f) 75
VISam 350 Ai93(TITL-GCaMP6f) 91
VISam 375 Ai93(TITL-GCaMP6f) 450
VISl 175 Ai148(TIT2L-GC6f-ICL-tTA2) 161
VISl 175 Ai93(TITL-GCaMP6f) 3445
VISl 250 Ai148(TIT2L-GC6f-ICL-tTA2) 83
VISl 275 Ai148(TIT2L-GC6f-ICL-tTA2) 235
VISl 275 Ai162(TIT2L-GC6s-ICL-tTA2) 139
VISl 275 Ai93(TITL-GCaMP6f) 5396
VISl 325 Ai93(TITL-GCaMP6f) 33
VISl 350 Ai93(TITL-GCaMP6f) 388
VISl 375 Ai148(TIT2L-GC6f-ICL-tTA2) 1888
VISl 375 Ai93(TITL-GCaMP6f) 955
VISl 400 Ai148(TIT2L-GC6f-ICL-tTA2) 176
VISl 550 Ai148(TIT2L-GC6f-ICL-tTA2) 502
VISl 570 Ai148(TIT2L-GC6f-ICL-tTA2) 35
VISl 625 Ai148(TIT2